# Transformers no PyTorch

Nesta seção, vamos criar modelos de transformadores usando a biblioteca `nn.torch`.


### Setup

In [1]:
def warn(*args, **kwargs):
    pass
    
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import importlib.util
import subprocess
import sys

def check_and_install(package, pip_name=None):
    if pip_name is None:
        pip_name = package
    spec = importlib.util.find_spec(package)
    if spec is None:
        print(f"{package} não está instalado. Instalando...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
    else:
        print(f"{package} já está instalado.")

In [2]:
# check_and_install('Levenshtein')
# check_and_install('torch','torch==2.3.0')
# check_and_install('torchtext','torchtext==0.18.0')

### Importar bibliotecas

In [3]:
import os
import sys
import time
from pathlib import Path
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import requests

from Levenshtein import distance
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

### Funções auxiliares

In [4]:
def plot_embdings(my_embdings,name,vocab):

  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')

  # Plot the data points
  ax.scatter(my_embdings[:,0], my_embdings[:,1], my_embdings[:,2])

  # Label the points
  for j, label in enumerate(name):
      i=vocab.get_stoi()[label]
      ax.text(my_embdings[j,0], my_embdings[j,1], my_embdings[j,2], label)

  # Set axis labels
  ax.set_xlabel('X Label')
  ax.set_ylabel('Y Label')
  ax.set_zlabel('Z Label')

  # Show the plot
  plt.show()

### Transformers
Este bloco de código cria uma instância do modelo Transformer a partir do módulo nn (rede neural) no PyTorch. O parâmetro nhead especifica o número de cabeças no mecanismo de atenção multicabeça, que é um componente crucial da arquitetura do Transformer. Neste caso, ele é definido como 16.

O parâmetro num_encoder_layers determina o número de camadas do codificador no modelo Transformer. Aqui, ele é definido como 12.


In [5]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)

Essas duas linhas criam tensores aleatórios para representar as sequências de origem e destino para o modelo Transformer.

`src` representa 10 sequências de origem, cada uma com um comprimento de 32 e uma dimensão de recurso de 512.
`tgt` representa 20 sequências de destino, cada uma com um comprimento de 32 e uma dimensão de recurso de 512.
No contexto de tarefas de sequência para sequência, as sequências de origem são os dados de entrada (por exemplo, frases em um idioma) e as sequências de destino são a saída desejada (por exemplo, as frases correspondentes em outro idioma).

In [6]:
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))

Em seguida, passe os tensores de origem e destino pelo modelo Transformer. A variável out conterá a saída do modelo Transformer, que deve ter o mesmo formato que o tensor tgt ((20, 32, 512)). Essa saída pode ser processada posteriormente ou usada para tarefas posteriores, como calcular uma função de perda para treinamento ou gerar texto para inferência.

In [7]:
out = transformer_model(src, tgt)

## Atenção MultiHead (multi-head attention)

`nn.MultiheadAttention` é um módulo no PyTorch que implementa o mecanismo de autoatenção multi-head, um componente-chave da arquitetura Transformer. Este mecanismo de atenção permite que o modelo se concentre em diferentes partes da sequência de entrada simultaneamente, capturando várias dependências contextuais e melhorando a capacidade do modelo de processar padrões complexos de linguagem natural.

O módulo `nn.MultiheadAttention` tem três entradas principais: `query`, `key` e `value`, conforme ilustrado abaixo.
<p style="text-align:center">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/MultiHeadAttention.png" width="300" alt="MultiHead"/>
</p>
O mecanismo de atenção multi-cabeça funciona primeiro dividindo as entradas `query`, `key` e `value` em várias "cabeças", cada uma com seu próprio conjunto de pesos aprendíveis. Esse processo permite que o modelo aprenda diferentes padrões de atenção em paralelo.

As saídas de todas as cabeças são concatenadas e passadas por uma camada linear, conhecida como projeção de saída, para combinar as informações aprendidas por cada cabeça. Essa saída final representa a sequência contextualmente enriquecida que pode ser usada em camadas subsequentes do modelo Transformer.

In [8]:
# Embedding dimension
embed_dim =4
# Number of attention heads
num_heads = 2
print("should be zero:",embed_dim %num_heads)

should be zero: 0


In [9]:
# Initialize MultiheadAttention
multihead_attn = nn.MultiheadAttention(embed_dim=embed_dim, 
                                       num_heads=num_heads,
                                       batch_first=False)

In [10]:
seq_length = 10 # Sequence length
batch_size = 5 # Batch size
query = torch.rand((seq_length, batch_size, embed_dim))
key = torch.rand((seq_length, batch_size, embed_dim))
value = torch.rand((seq_length, batch_size, embed_dim))

In [11]:
# Perform multi-head attention
attn_output, _= multihead_attn(query, key, value)
print("Attention Output Shape:", attn_output.shape)

Attention Output Shape: torch.Size([10, 5, 4])


### TransformerEncoderLayer e TransformerEncoder

O `TransformerEncoderLayer` e o `TransformerEncoder` são componentes essenciais da arquitetura do Transformer no PyTorch. Esses componentes trabalham juntos para criar uma rede neural multicamadas baseada em atenção.

### TransformerEncoderLayer:
Esta é uma única camada de codificação na arquitetura do Transformer, consistindo em duas subcamadas primárias, conforme mostrado abaixo: a camada Multi-head Self-Attention e a Feed-Forward Network (FFN). Cada uma dessas subcamadas é seguida por uma conexão residual e normalização de camada.
<p style="text-align:center">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/TrLayer.png" width="200" alt="TrLayer"/>
</p>

### TransformerEncoder:
O TransformerEncoder é uma pilha de múltiplas instâncias `TransformerEncoderLayer`. O codificador consiste em N camadas idênticas. N pode ser ajustado com base na complexidade desejada do modelo.

O codificador pega uma sequência de entrada, aplica codificação posicional e a passa por cada um dos TransformerEncoderLayers sequencialmente. Isso permite que o modelo aprenda representações ricas e hierárquicas da sequência de entrada, capturando dependências locais e de longo alcance.

O TransformerEncoder aceita os seguintes parâmetros:
- `src` (obrigatório): A sequência para o codificador.

- `mask` (opcional): O parâmetro mask é usado para restringir o mecanismo de atenção de considerar certas posições na sequência de entrada. É um tensor binário com o mesmo formato da sequência de entrada. Um valor de 1 indica que a atenção é permitida, enquanto um valor de 0 indica que a atenção deve ser desconsiderada. Esta máscara é particularmente útil ao trabalhar com máscaras de atenção triangulares, onde cada posição na sequência pode atender apenas às posições anteriores.

- `src_key_padding_mask` (opcional): O parâmetro src_key_padding_mask é usado para especificar quais posições na sequência de entrada correspondem aos tokens de preenchimento. É um tensor binário com formato (batch_size, sequence_length). Um valor de 1 indica que a posição correspondente contém um token válido, enquanto um valor de 0 indica que a posição contém um token de preenchimento. Ao fornecer essa máscara, o mecanismo de atenção pode ignorar tokens de preenchimento e focar apenas nas partes significativas da sequência de entrada. Esse parâmetro é particularmente útil ao lidar com sequências de comprimento variável que foram preenchidas para um comprimento fixo.

In [12]:
# Embedding dimension
embed_dim = 4

# Number of attention h
num_heads = 2

# Checking if the embedding dimension is divisible by the number of heads, print("should be zero", embed_dim % num_h
# Number of encoder layers
num_layers = 6

# Initialize the encoder layer with specified embedding dimension and number of heads.
encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)

# Build the transformer encoder by stacking the encoder layer 6 times.
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

In [13]:
# Define sequence length as 10 and batch size as 5 for the input data.
seq_length = 10 # Sequence length
batch_size = 5 # Batch size

# Generate random input tensor to simulate input embeddings for the transformer encoder.
x = torch.rand((seq_length, batch_size, embed_dim))

# Apply the transformer encoder to the input
encoded = transformer_encoder(x)

# Output the shape of the encoded tensor to verify the transformation.
print("Encoded Tensor Shape:", encoded.shape)

Encoded Tensor Shape: torch.Size([10, 5, 4])


## Exercício
Criar um transformador multi-hear e o usar para codificar um vetor de entrada. Para conseguir isso, você aproveitará nn.TransformerEncoderLayer e nn.TransformerEncoder.

>Observe que o `tamanho de incorporação` deve ser divisível pelo `número de cabeças de atenção`.

1. **Crie um codificador de transformador com os seguintes parâmetros:**
- `tamanho de incorporação` = 240
- `número de camadas` = 12
- `número de cabeças de atenção` = 12

2. **Crie um tensor de entrada aleatório de comprimento 20 e tamanho de lote de 1**

3. **Passe o tensor de entrada para modelar e imprima o formato de sua saída**

In [14]:
embed_dim = 240
num_heads = 12
num_layers = 12
encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

seq_length = 20
batch_size = 1
x = torch.rand((seq_length, batch_size, embed_dim))
encoded = transformer_encoder(x)
print("Encoded Tensor Shape:", encoded.shape)

Encoded Tensor Shape: torch.Size([20, 1, 240])
